<a href="https://colab.research.google.com/github/amanmaurya7/sentinel-1/blob/main/sentinel_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import json
from shapely.geometry import shape
from datetime import datetime
from google.colab import files

# Chennai bounding box (min_lon, min_lat, max_lon, max_lat)
bbox = [80.0837, 12.8347, 80.4577, 13.2677]

# STAC API endpoint
stac_url = "https://catalogue.dataspace.copernicus.eu/stac/search"

# Request latest Sentinel-1 products
body = {
    "collections": ["SENTINEL-1"],
    "bbox": bbox,
    "datetime": "2023-01-01T00:00:00Z/2025-12-31T23:59:59Z",
    "limit": 100
}

response = requests.post(stac_url, json=body)
response.raise_for_status()
data = response.json()

# Sort by datetime descending
features = sorted(
    data.get("features", []),
    key=lambda f: f["properties"]["datetime"],
    reverse=True
)

# Function to get weather from Open-Meteo
def get_weather(lat, lon, date):
    url = f"https://archive-api.open-meteo.com/v1/archive?latitude={lat}&longitude={lon}&start_date={date}&end_date={date}&hourly=temperature_2m,cloudcover,precipitation,wind_speed_10m"
    try:
        res = requests.get(url).json()
        if "hourly" in res:
            hourly = res["hourly"]
            avg_temp = sum(hourly["temperature_2m"])/len(hourly["temperature_2m"])
            avg_cloud = sum(hourly["cloudcover"])/len(hourly["cloudcover"])
            total_precip = sum(hourly["precipitation"])
            avg_wind = sum(hourly["wind_speed_10m"])/len(hourly["wind_speed_10m"])
            return {
                "temperature_c": round(avg_temp,1),
                "cloud_coverage_percent": round(avg_cloud,1),
                "precipitation_mm": round(total_precip,1),
                "wind_speed_m_s": round(avg_wind,1)
            }
        else:
            return None
    except:
        return None

# Function to compute disaster risk labels for ML
def compute_disaster_labels(weather):
    if not weather:
        return {
            "flood_risk": None,
            "cyclone_risk": None,
            "heatwave_risk": None,
            "drought_risk": None,
            "wildfire_risk": None,
            "landslide_risk": None
        }
    flood = 2 if weather["precipitation_mm"] > 100 else (1 if weather["precipitation_mm"] > 50 else 0)
    cyclone = 2 if weather["wind_speed_m_s"] > 25 else (1 if weather["wind_speed_m_s"] > 15 else 0)
    heatwave = 2 if weather["temperature_c"] > 40 else (1 if weather["temperature_c"] > 35 else 0)
    drought = 2 if weather["precipitation_mm"] < 2 else (1 if weather["precipitation_mm"] < 5 else 0)
    wildfire = 2 if weather["temperature_c"] > 40 and weather["precipitation_mm"] < 5 else (1 if weather["temperature_c"] > 35 and weather["precipitation_mm"] < 10 else 0)
    landslide = 2 if weather["precipitation_mm"] > 120 else (1 if weather["precipitation_mm"] > 80 else 0)
    return {
        "flood_risk": flood,
        "cyclone_risk": cyclone,
        "heatwave_risk": heatwave,
        "drought_risk": drought,
        "wildfire_risk": wildfire,
        "landslide_risk": landslide
    }

# Build dataset for ML
dataset = []
for idx, feat in enumerate(features, start=1):
    props = feat["properties"]
    geom = shape(feat["geometry"]).bounds
    minx, miny, maxx, maxy = geom
    center_lat = (miny + maxy)/2
    center_lon = (minx + maxx)/2
    acq_date = props.get("datetime", "").split("T")[0]

    weather = get_weather(center_lat, center_lon, acq_date)
    labels = compute_disaster_labels(weather)

    record = {
        "record_id": idx,
        "acquisition_datetime_utc": props.get("datetime"),
        "center_lat": center_lat,
        "center_lon": center_lon,
        "bbox_north": maxy,
        "bbox_south": miny,
        "bbox_east": maxx,
        "bbox_west": minx,
        "temperature_c": weather.get("temperature_c") if weather else None,
        "precipitation_mm": weather.get("precipitation_mm") if weather else None,
        "wind_speed_m_s": weather.get("wind_speed_m_s") if weather else None,
        "cloud_coverage_percent": weather.get("cloud_coverage_percent") if weather else None,
        **labels
    }
    dataset.append(record)

# Save dataset as JSON for ML
file_name = "sentinel1_chennai_disaster_dataset.json"
with open(file_name, "w") as f:
    json.dump(dataset, f, indent=2)

# Download in Colab
files.download(file_name)

print(f"✅ {len(dataset)} records generated for disaster prediction and downloaded as {file_name}")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ 100 records generated for disaster prediction and downloaded as sentinel1_chennai_disaster_dataset.json


In [3]:
import requests
import json
from shapely.geometry import shape
from datetime import datetime
from google.colab import files

# Chennai bounding box (min_lon, min_lat, max_lon, max_lat)
bbox = [80.0837, 12.8347, 80.4577, 13.2677]

# STAC API endpoint
stac_url = "https://catalogue.dataspace.copernicus.eu/stac/search"

# Correct ISO 8601 datetime format for recent data
body = {
    "collections": ["SENTINEL-1"],
    "bbox": bbox,
    "datetime": "2023-01-01T00:00:00Z/2023-01-31T23:59:59Z",
    "limit": 100
}

# Fetch Sentinel-1 metadata
response = requests.post(stac_url, json=body)
response.raise_for_status()
data = response.json()

# Save raw Sentinel-1 response as JSON
raw_response_file_name = "sentinel1_raw_response.json"
with open(raw_response_file_name, "w") as f:
    json.dump(data, f, indent=2)

# Sort features by datetime descending (latest first)
features = sorted(
    data.get("features", []),
    key=lambda f: f["properties"]["datetime"],
    reverse=True
)

# Function to get weather from Open-Meteo
def get_weather(lat, lon, date):
    url = f"https://archive-api.open-meteo.com/v1/archive?latitude={lat}&longitude={lon}&start_date={date}&end_date={date}&hourly=temperature_2m,cloudcover,precipitation,wind_speed_10m"
    try:
        res = requests.get(url).json()
        if "hourly" in res:
            hourly = res["hourly"]
            avg_temp = sum(hourly["temperature_2m"]) / len(hourly["temperature_2m"])
            avg_cloud = sum(hourly["cloudcover"]) / len(hourly["cloudcover"])
            total_precip = sum(hourly["precipitation"])
            avg_wind = sum(hourly["wind_speed_10m"]) / len(hourly["wind_speed_10m"])
            return {
                "temperature_c": round(avg_temp,1),
                "cloud_coverage_percent": round(avg_cloud,1),
                "precipitation_mm": round(total_precip,1),
                "wind_speed_m_s": round(avg_wind,1)
            }
        else:
            return None
    except:
        return None

# Function to calculate disaster risk levels
def disaster_risks(weather):
    if not weather:
        return {
            "flood_risk_level": None,
            "cyclone_risk_level": None,
            "heatwave_risk_level": None,
            "drought_risk_level": None,
            "wildfire_risk_level": None,
            "landslide_risk_level": None
        }
    flood = "High" if weather["precipitation_mm"] > 100 else ("Medium" if weather["precipitation_mm"] > 50 else "Low")
    cyclone = "High" if weather["wind_speed_m_s"] > 25 else ("Medium" if weather["wind_speed_m_s"] > 15 else "Low")
    heatwave = "High" if weather["temperature_c"] > 40 else ("Medium" if weather["temperature_c"] > 35 else "Low")
    drought = "High" if weather["precipitation_mm"] < 2 else ("Medium" if weather["precipitation_mm"] < 5 else "Low")
    wildfire = "High" if weather["temperature_c"] > 40 and weather["precipitation_mm"] < 5 else ("Medium" if weather["temperature_c"] > 35 and weather["precipitation_mm"] < 10 else "Low")
    landslide = "High" if weather["precipitation_mm"] > 120 else ("Medium" if weather["precipitation_mm"] > 80 else "Low")
    return {
        "flood_risk_level": flood,
        "cyclone_risk_level": cyclone,
        "heatwave_risk_level": heatwave,
        "drought_risk_level": drought,
        "wildfire_risk_level": wildfire,
        "landslide_risk_level": landslide
    }

# Build structured records
records = []
for idx, feat in enumerate(features, start=1):
    props = feat["properties"]
    geom = shape(feat["geometry"]).bounds
    minx, miny, maxx, maxy = geom
    center_lat = (miny + maxy) / 2
    center_lon = (minx + maxx) / 2
    acq_date = props.get("datetime", "").split("T")[0]

    weather = get_weather(center_lat, center_lon, acq_date)
    risks = disaster_risks(weather)

    record = {
        "record_id": idx,
        "product_id": feat["id"],
        "satellite": props.get("platform", "Sentinel-1"),
        "constellation_status": "Single (S1A/S1B) or Combined",
        "acquisition_datetime_utc": props.get("datetime"),
        "orbit_parameters": {
            "orbit_number": props.get("orbitNumber"),
            "orbit_direction": props.get("orbitDirection"),
            "track": props.get("relativeOrbitNumber"),
            "frame": props.get("sliceNumber")
        },
        "sensor_parameters": {
            "instrument_mode": props.get("sar:instrument_mode"),
            "product_type": props.get("productType"),
            "polarization": props.get("polarizations"),
            "swath_id": props.get("sar:swath"),
            "resolution_m": props.get("gsd", 10)
        },
        "geographic_extent": {
            "center_lat": center_lat,
            "center_lon": center_lon,
            "bbox_north": maxy,
            "bbox_south": miny,
            "bbox_east": maxx,
            "bbox_west": minx
        },
        "data_quality": {
            "file_size_mb": None,
            "processing_level": props.get("processingLevel"),
            "calibration_applied": True,
            "noise_equivalent_sigma_zero_db": None
        },
        "environmental_context": {
            "weather_conditions": weather,
            **risks,
            "disaster_event": None
        },
        "data_access": {
            "availability_copernicus": True,
            "availability_gee": False,
            "processing_baseline": props.get("processingBaseline"),
            "download_link": next((l["href"] for l in feat["assets"].values() if l.get("href")), None)
        }
    }
    records.append(record)

# Save JSON and download in Colab
file_name = "sentinel1_chennai_latest_disaster.json"
with open(file_name, "w") as f:
    json.dump(records, f, indent=2)

files.download(file_name)
files.download(raw_response_file_name)

print(f"✅ {len(records)} latest Sentinel-1 records with disaster context saved and downloaded.")
print(f"✅ Raw Sentinel-1 response saved and downloaded as {raw_response_file_name}.")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ 20 latest Sentinel-1 records with disaster context saved and downloaded.
✅ Raw Sentinel-1 response saved and downloaded as sentinel1_raw_response.json.
